In [ ]:


###########import packages##########
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import ensemble
from sklearn.tree import ExtraTreeRegressor
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
import shap
import joblib
import warnings
from sklearn.metrics import r2_score
%matplotlib inline
###########wrapping root mean square error for later calls##########
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
###########loading data##########
fdata=pd.read_csv('finaldatabase.csv',encoding="utf-8")
raw_data=fdata.loc[:,[                     
    'xN2a_in0',#0
    'RH_CC',#1
    'dela_CH (mm)',#2
    'dela_M (um)',#3
    'dela_GDL (um)',#4
    'p_A (atm)',#5
    'T_cc (K)',#6
    'E cell (V)',#7
    'current',#8
    'fyN2',#9
                        ]]
###########train test splitting##########
standardized_data = (raw_data-np.mean(raw_data,axis=0))/np.std(raw_data,axis=0)
raw_input=raw_data.iloc[:,0:8]
raw_output=raw_data.iloc[:,9]*1e15
###########fix random seed for reproducability##########
seed=1
X_train, X_test, y_train, y_test = train_test_split(raw_input, raw_output, test_size=.15,random_state=seed)
y_test_values=y_test.values.astype(np.float32)


In [ ]:


ada_model=joblib.load('ada_model')
algorithm_name='AdaBoost'
result = ada_model.predict(X_test)
x_prediction=result
y_real=y_test_values
x_prediction_series=pd.Series(x_prediction)
y_real_series=pd.Series(y_real)
###########evaluating the regression quality##########

rmse_val= rmse(x_prediction,y_real)
r2_score_current=r2_score(x_prediction,y_real)
print('可决系数 r2_score_current: ',r2_score_current)
print('方差 rmse_val：',rmse_val)

###########generating a figure##########
x_y_x=np.arange(2,8,0.1)
x_y_y=np.arange(2,8,0.1)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(x_prediction,y_real,edgecolor='black',s=10,label=algorithm_name)
ax.plot(x_y_x,x_y_y)
plt.legend()
plt.xlabel(u"predict crossover rate ×10^15 mol/m/s/pa")
plt.ylabel(u"real predict crossover rate ×10^15 mol/m/s/pa")
plt.savefig('%s Crossover rate Regression.jpg' %algorithm_name,bbox_inches = 'tight')
plt.show()
name_list=list(X_test)
plt.bar(range(len(ada_model.feature_importances_)), ada_model.feature_importances_,edgecolor='black',tick_label=name_list)
plt.xticks(rotation=90)
plt.savefig('%s FeatureImportance.jpg' %algorithm_name,bbox_inches = 'tight')
plt.show()
print(ada_model.feature_importances_)



In [ ]:
##shapvalues caculation

explainer=shap.Explainer(ada_model)
shapvalues=explainer(X_test)
shap_values=explainer.shap_values(X_test)
shap_interaction_values = explainer.shap_interaction_values(X_test)
expected_value = explainer.expected_value
explainer2 = shap.Explainer(ada_model,X_test)
shap_values2 = explainer(X_test)

In [ ]:

##bar plot
shap.plots.bar(shapvalues[0],max_display=10,show=False)
plt.savefig('%s shapvalues_distribution_single.jpg' %algorithm_name,bbox_inches = 'tight')
plt.show()

##bar plot
shap.plots.bar(shapvalues,max_display=10,show=False)
plt.savefig('%s shapvalues_distribution.jpg' %algorithm_name,bbox_inches = 'tight')
plt.show()
print(name_list)
print(shapvalues.abs.mean(0).values)

##bar plot   optimally separate the SHAP values of the instances using a sklearn DecisionTreeRegressor. 
## 2 
shap.plots.bar(shapvalues.cohorts(2).abs.mean(0),show=False)
plt.savefig('%s shapvalues_distribution2.jpg' %algorithm_name,bbox_inches = 'tight')
plt.show()

##bar plot   optimally separate the SHAP values of the instances using a sklearn DecisionTreeRegressor. 
## 3 
shap.plots.bar(shapvalues.cohorts(3).abs.mean(0),show=False)
plt.savefig('%s shapvalues_distribution3.jpg' %algorithm_name,bbox_inches = 'tight')
plt.show()

##bar plot   optimally separate the SHAP values of the instances using a sklearn DecisionTreeRegressor. 
## 4 
shap.plots.bar(shapvalues.cohorts(4).abs.mean(0),show=False)
plt.savefig('%s shapvalues_distribution4.jpg' %algorithm_name,bbox_inches = 'tight')
plt.show()

##bar plot   optimally separate the SHAP values of the instances using a sklearn DecisionTreeRegressor. 
## 5 
shap.plots.bar(shapvalues.cohorts(5).abs.mean(0),show=False)
plt.savefig('%s shapvalues_distribution5.jpg' %algorithm_name,bbox_inches = 'tight')
plt.show()

##high low feature imortance distribution
j=0
ka = ["low {}".format(name_list[j]) if shapvalues.data[i,j] <= 0.35 else "high {}".format(name_list[j]) for i in range(shapvalues.shape[0])]
shap.plots.bar(shapvalues.cohorts(ka).abs.mean(0),show=False)
plt.savefig('%s shapvalues_distribution{}.jpg'.format(name_list[j]) %algorithm_name,bbox_inches = 'tight')
plt.show()

j=1
ka = ["low {}".format(name_list[j]) if shapvalues.data[i,j] <= 0.5 else "high {}".format(name_list[j]) for i in range(shapvalues.shape[0])]
shap.plots.bar(shapvalues.cohorts(ka).abs.mean(0),show=False)
plt.savefig('%s shapvalues_distribution{}.jpg'.format(name_list[j]) %algorithm_name,bbox_inches = 'tight')
plt.show()

j=2
ka = ["low {}".format(name_list[j]) if shapvalues.data[i,j] <= 1.5 else "high {}".format(name_list[j]) for i in range(shapvalues.shape[0])]
shap.plots.bar(shapvalues.cohorts(ka).abs.mean(0),show=False)
plt.savefig('%s shapvalues_distribution{}.jpg'.format(name_list[j]) %algorithm_name,bbox_inches = 'tight')
plt.show()

j=3
ka = ["low {}".format(name_list[j]) if shapvalues.data[i,j] <= 250 else "high {}".format(name_list[j]) for i in range(shapvalues.shape[0])]
shap.plots.bar(shapvalues.cohorts(ka).abs.mean(0),show=False)
plt.savefig('%s shapvalues_distribution{}.jpg'.format(name_list[j]) %algorithm_name,bbox_inches = 'tight')
plt.show()

j=4
ka = ["low {}".format(name_list[j]) if shapvalues.data[i,j] <= 250 else "high {}".format(name_list[j]) for i in range(shapvalues.shape[0])]
shap.plots.bar(shapvalues.cohorts(ka).abs.mean(0),show=False)
plt.savefig('%s shapvalues_distribution{}.jpg'.format(name_list[j]) %algorithm_name,bbox_inches = 'tight')
plt.show()

j=5
ka = ["low {}".format(name_list[j]) if shapvalues.data[i,j] <= 3 else "high {}".format(name_list[j]) for i in range(shapvalues.shape[0])]
shap.plots.bar(shapvalues.cohorts(ka).abs.mean(0),show=False)
plt.savefig('%s shapvalues_distribution{}.jpg'.format(name_list[j]) %algorithm_name,bbox_inches = 'tight')
plt.show()

j=6
ka = ["low {}".format(name_list[j]) if shapvalues.data[i,j] <= 340 else "high {}".format(name_list[j]) for i in range(shapvalues.shape[0])]
shap.plots.bar(shapvalues.cohorts(ka).abs.mean(0),show=False)
plt.savefig('%s shapvalues_distribution{}.jpg'.format(name_list[j]) %algorithm_name,bbox_inches = 'tight')
plt.show()

j=7
ka = ["low {}".format(name_list[j]) if shapvalues.data[i,j] <= 0.6 else "high {}".format(name_list[j]) for i in range(shapvalues.shape[0])]
shap.plots.bar(shapvalues.cohorts(ka).abs.mean(0),show=False)
plt.savefig('%s shapvalues_distribution{}.jpg'.format(name_list[j]) %algorithm_name,bbox_inches = 'tight')
plt.show()

##heatmap
shap.plots.heatmap(shapvalues[:1000],show=False)
plt.savefig('%s shapvalues_heatmap.jpg' %algorithm_name,bbox_inches = 'tight')
plt.show()

shap.plots.heatmap(shapvalues[:1000], feature_values=shapvalues.abs.max(0),show=False)
plt.savefig('%s shapvalues_heatmap_orderby_max.jpg' %algorithm_name,bbox_inches = 'tight')
plt.show()

#order by sum shapvalues
shap.plots.heatmap(shapvalues[:1000], instance_order=shapvalues.sum(1)[:1000],show=False)
plt.savefig('%s shapvalues_heatmap_orderby_sum.jpg' %algorithm_name,bbox_inches = 'tight')
plt.show()

##decision plot
shap.decision_plot(expected_value, shap_values[:10], X_test,show=False,highlight=0)
plt.savefig('%s decision_plot10.jpg' %algorithm_name,bbox_inches = 'tight')
plt.show()

##decision plot
shap.decision_plot(expected_value, shap_values[0], X_test,show=False,highlight=0)
plt.savefig('%s decision_plot10_0.jpg' %algorithm_name,bbox_inches = 'tight')
plt.show()



for i in range(len(list(X_test))):
    shap.dependence_plot("%s"%list(X_test)[i], shap_values[:2000], X_test[:2000],show=False)
    plt.savefig('%s dependence_plot_%s.jpg' %(algorithm_name,list(X_test)[i]),bbox_inches = 'tight')

    
shap.plots.beeswarm(shap_values2.abs, color="shap_red",show=False)
plt.savefig('%s beenswarm_abs.jpg' %algorithm_name,bbox_inches = 'tight')
plt.show()

## summarize the effects of all the features
shap.summary_plot(shap_values[:500], X_test[:500],show=False)
plt.savefig('%s feature_effect.jpg' %algorithm_name,bbox_inches = 'tight')
plt.show()



shap.plots.beeswarm(shap_values2.abs, color="shap_red",show=False)



In [ ]:
    
##identify typical prediction pahths
y_pred = ada_model.predict(X_test)

T = X_test[(y_pred >= 0)&(y_pred <= 2.5)]
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    sh = explainer.shap_values(T)[:500]
shap.decision_plot(expected_value, sh, T, feature_order='hclust',show=False)

T = X_test[(y_pred >= 3)&(y_pred <= 3.1)]
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    sh = explainer.shap_values(T)[:500]
shap.decision_plot(expected_value, sh, T, feature_order='hclust',show=False)

T = X_test[(y_pred >= 4)&(y_pred <= 4.1)]
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    sh = explainer.shap_values(T)[:500]
shap.decision_plot(expected_value, sh, T, feature_order='hclust',show=False)

T = X_test[(y_pred >= 5)&(y_pred <= 5.1)]
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    sh = explainer.shap_values(T)[:500]
shap.decision_plot(expected_value, sh, T, feature_order='hclust',show=False)

T = X_test[(y_pred >= 6)&(y_pred <= 6.1)]
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    sh = explainer.shap_values(T)[:500]
shap.decision_plot(expected_value, sh, T, feature_order='hclust',show=False)

T = X_test[(y_pred >= 7)&(y_pred <= 7.1)]
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    sh = explainer.shap_values(T)[:500]
shap.decision_plot(expected_value, sh, T, feature_order='hclust',show=False)




plt.savefig('%s tupical_prediction_path500.jpg' %algorithm_name,bbox_inches = 'tight')
plt.show()


In [ ]:
try:

    ##shap_interaction_values
    shap.decision_plot(expected_value, shap_interaction_values[:10], X_test[:10],show=False)
    plt.savefig('%s decision_plot10_interactionvalues.jpg' %algorithm_name,bbox_inches = 'tight')
    plt.show()

    shap.decision_plot(expected_value, shap_interaction_values[:10], X_test[:10],show=False,feature_display_range=slice(None, None, -1))
    plt.savefig('%s decision_plot10_interactionvalues_long.jpg' %algorithm_name,bbox_inches = 'tight')
    plt.show()

    shap.decision_plot(expected_value, shap_interaction_values[:1], X_test[:1],show=False,highlight=0)
    plt.savefig('%s decision_plot10_interactionvalues_0.jpg' %algorithm_name,bbox_inches = 'tight')
    plt.show()

    shap.summary_plot(shap_interaction_values[:1000], X_test[:1000],show=False)
    plt.savefig('%s effect_betweenfeatures.jpg' %algorithm_name,bbox_inches = 'tight')
    plt.show()


    for i in range(len(name_list)):
        for j in range(len(name_list)):
            shap.dependence_plot(
                (name_list[i], name_list[j]),
                shap_interaction_values, X_test,
                show=False

            )
            plt.savefig('%s dependence_interactionplot_%s%s.jpg' %(algorithm_name,list(X_test)[i],name_list[j]),bbox_inches = 'tight')



    import matplotlib.pylab as pl
    import numpy as np

    tmp = np.abs(shap_interaction_values).sum(0)

    for i in range(tmp.shape[0]):
        tmp[i,i] = 0
    inds = np.argsort(-tmp.sum(0))[:50]
    tmp2 = tmp[inds,:][:,inds]
    pl.figure(figsize=(8,8))
    pl.imshow(tmp2)
    pl.yticks(range(tmp2.shape[0]), X_test.columns[inds], rotation=50.4, horizontalalignment="right")
    pl.xticks(range(tmp2.shape[0]), X_test.columns[inds], rotation=50.4, horizontalalignment="left")
    pl.gca().xaxis.tick_top()
    pl.show()
    
except:
    print('shap_interaction_values is not defined')

In [ ]:


###########import packages##########
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import ensemble
from sklearn.tree import ExtraTreeRegressor
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
import shap
import joblib
import warnings
from sklearn.metrics import r2_score
%matplotlib inline
###########wrapping root mean square error for later calls##########
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
###########loading data##########
fdata=pd.read_csv('finaldatabase.csv',encoding="utf-8")
raw_data=fdata.loc[:,[                     
    'xN2a_in0',#0
    'RH_CC',#1
    'dela_CH (mm)',#2
    'dela_M (um)',#3
    'dela_GDL (um)',#4
    'p_A (atm)',#5
    'T_cc (K)',#6
    'E cell (V)',#7
    'current',#8
    'fyN2',#9
                        ]]
###########train test splitting##########
standardized_data = (raw_data-np.mean(raw_data,axis=0))/np.std(raw_data,axis=0)
raw_input=raw_data.iloc[:,0:8]
raw_output=raw_data.iloc[:,9]*1e15
###########fix random seed for reproducability##########
seed=1
X_train, X_test, y_train, y_test = train_test_split(raw_input, raw_output, test_size=.15,random_state=seed)
y_test_values=y_test.values.astype(np.float32)


In [ ]:


ada_model=joblib.load('ada_model')
algorithm_name='RandomForest'
result = ada_model.predict(X_test)
x_prediction=result
y_real=y_test_values
x_prediction_series=pd.Series(x_prediction)
y_real_series=pd.Series(y_real)
###########evaluating the regression quality##########

rmse_val= rmse(x_prediction,y_real)
r2_score_current=r2_score(x_prediction,y_real)
print('可决系数 r2_score_current: ',r2_score_current)
print('方差 rmse_val：',rmse_val)

###########generating a figure##########
x_y_x=np.arange(2,8,0.1)
x_y_y=np.arange(2,8,0.1)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(x_prediction,y_real,edgecolor='black',s=10,label=algorithm_name)
ax.plot(x_y_x,x_y_y)
plt.legend()
plt.xlabel(u"predict current A/cm^2")
plt.ylabel(u"real predict current A/cm^2")
plt.savefig('%s CurrentRegression.jpg' %algorithm_name,bbox_inches = 'tight')
plt.show()
name_list=list(X_test)
plt.bar(range(len(ada_model.feature_importances_)), ada_model.feature_importances_,edgecolor='black',tick_label=name_list)
plt.xticks(rotation=90)
plt.savefig('%s FeatureImportance.jpg' %algorithm_name,bbox_inches = 'tight')
plt.show()
print(ada_model.feature_importances_)

